In [1]:
from pathlib import Path
import cv2
import numpy as np
from mplex import Grid
from bino_utils.plotting import add_ocular_condition_symbol

In [2]:
data_dir = Path("../../data/lensectomy")
out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)
fmt = ".pdf"

centers = {
    "le": (440 + 67j, 440 + 212j),
    "re": (439 + 20j, 437 + 155j),
    "sham": (443 + 65j, 441 + 210j),
    "null": (435 + 73j, 431 + 210j),
}

w_new, h_new = 144, 144
dst = np.array([[w_new / 2, 0], [w_new / 2, h_new], [0, 0]], dtype=np.float32)
q0 = 72
q1 = 72 + 144j
q2 = q0 + (q1 - q0) * 1j
q = np.array([(i.real, i.imag) for i in [q0, q1, q2]], dtype=np.float32)

In [ ]:
g = Grid(60, (1, 4), space=8)

for ax, (condition, (p0, p1)) in zip(g.axs.ravel(), centers.items()):
    im = cv2.imread(str(data_dir / f"images/{condition}.png"), cv2.IMREAD_GRAYSCALE)
    p2 = p0 + (p1 - p0) * 1j
    p = np.array([(i.real, i.imag) for i in [p0, p1, p2]], dtype=np.float32)
    M = cv2.getAffineTransform(p, q)
    im = cv2.warpAffine(im, M, (w_new, h_new))
    ax.imshow(im, vmin=50, vmax=255, cmap="gray")

    ax.axis("off")
    add_ocular_condition_symbol(condition, ax=ax, delense=True, scale=0.75, ypad=1)

if fmt:
    g.savefig((out_dir / "lensectomy_for_each_condition").with_suffix(fmt), dpi=300)